In [1]:
import os
import sys
from IPython import get_ipython
from pathlib import Path
import json
import pandas as pd
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
from itertools import combinations
import traceback 

In [2]:
def get_script_directory():
    """
    Returns the ACTUAL directory containing the notebook/script.
    Works in:
    - VS Code Jupyter notebooks
    - Regular Jupyter Notebook/Lab
    - Standalone Python scripts
    """
    # If running in Jupyter
    if 'ipykernel' in sys.modules:
        try:
            # 1. First try VS Code's special attribute
            shell = get_ipython()
            if hasattr(shell, '__vsc_ipynb_file__'):
                return str(Path(shell.__vsc_ipynb_file__).parent)
            
            # 2. Try Jupyter notebook path (modern Jupyter)
            from notebook.notebookapp import list_running_servers
            servers = list_running_servers()
            if servers:
                import requests
                from urllib.parse import urljoin
                kernel_id = Path(get_ipython().config['IPKernelApp']['connection_file']).stem.replace('kernel-', '')
                for server in servers:
                    sessions = requests.get(urljoin(server['url'], 'api/sessions'), params={'token': server.get('token', '')}).json()
                    for session in sessions:
                        if session['kernel']['id'] == kernel_id:
                            return str(Path(server['notebook_dir']) / Path(session['notebook']['path']).parent)
            
            # 3. Fallback to current working directory
            return str(Path.cwd())
        except:
            return str(Path.cwd())
    
    # If running as a Python script
    return str(Path(__file__).parent.resolve())

In [3]:
# Function to delete all files and folders in path except .obsidian
def delete_all_except_obsidian(path):
    """
    Deletes all files and folders in the given path except for the .obsidian folder.
    """
    for item in os.listdir(path):
        item_path = os.path.join(path, item)
        if os.path.isdir(item_path) and item != '.obsidian':
            # Recursively delete the contents of the directory
            for root, dirs, files in os.walk(item_path, topdown=False):
                for file in files:
                    os.remove(os.path.join(root, file))
                for dir in dirs:
                    os.rmdir(os.path.join(root, dir))
            os.rmdir(item_path)  # Remove the now-empty directory
        elif os.path.isfile(item_path):
            os.remove(item_path)

In [4]:
def load_json(json_path):
    with open(json_path, 'r') as file:
        json_contents = json.load(file)
    return json_contents

In [5]:
def construct_url_dict(gid_dict, url_template):
    url_dict = {}
    for sheet, gid in gid_dict.items():
        full_url = url_template.replace("edit?gid=gid_value#gid=gid_value", f"export?format=csv&gid={gid}")
        url_dict[sheet] = full_url
    return url_dict

In [6]:
def construct_master_url_dict(sheets_dict):
    master_url_dict={}
    for spreadsheet, spreadsheet_dict in sheets_dict.items():
        spreadsheet_gid_dict=spreadsheet_dict['sheets']
        spreadsheet_url_template=spreadsheet_dict['link_template']
        spreadsheet_url_dict=construct_url_dict(spreadsheet_gid_dict, spreadsheet_url_template)
        master_url_dict[spreadsheet]=spreadsheet_url_dict
    return master_url_dict

In [7]:
def row_first_value(row):
    first_value = str(row.iloc[0]).strip() if pd.notna(row.iloc[0]) else None
    return first_value

def first_column_indecies(df):
    indecies_list = df.iloc[1:, 0]
    return indecies_list

In [ ]:
def initial_content_dict_from_url(url):
    """
    Reads a CSV file and constructs a dictionary where each row's title (first column)
    is the key, and the rest of the row's data is a dictionary of key-value pairs.
    
    note: handles duplicate column names and row titles by grouping all rows with the same title 
    and adding a suffix to the column names.

    Args:
        url (str): The URL or path to the CSV file.

    Returns:
        dict: A dictionary with row titles as keys and row data as nested dictionaries.
    """
    df = pd.read_csv(url)
    
    # Convert float columns to Int64 where appropriate
    for col in df.select_dtypes(include=['float64']):
        if (df[col].dropna().apply(float.is_integer).all()):
            df[col] = df[col].astype('Int64')
    
    content_dict = {}
    
    for _, row in df.iterrows():
        if pd.notna(row.iloc[0]):  # First column as key
            index_value = row.iloc[0]
            
            if index_value not in content_dict:
                content_dict[index_value] = {}
            
            # Process each column
            for i, val in enumerate(row.iloc[1:]):
                if pd.notna(val):
                    original_col = df.columns[i+1]
                    base_col = original_col.split('.')[0]  # Remove pandas suffixes
                    
                    # Find next available column name
                    col_name = base_col
                    suffix = 1
                    while col_name in content_dict[index_value]:
                        suffix += 1
                        col_name = f"{base_col}_{suffix}"
                    
                    content_dict[index_value][col_name] = val
    
    return content_dict

In [ ]:
# def extract_filename(path: str) -> str:
#     # Split by forward slash and filter out empty strings (this handles multiple slashes)
#     parts = [part for part in path.split('/') if part]
#     # Return the last element, if available; otherwise return an empty string.
#     return parts[-1] if parts else ''

In [ ]:
# def extract_directory(path: str) -> str:
#     # Remove any trailing slashes so that they don't interfere with finding the last meaningful slash.
#     stripped_path = path.rstrip('/')
#     # Find the index of the last slash in the stripped path.
#     last_slash_index = stripped_path.rfind('/')
#     # If a slash exists, return everything before it. If not, return an empty string.
#     return stripped_path[:last_slash_index] if last_slash_index != -1 else ''


In [11]:
# def remove_empty_lines(s: str) -> str:
#     """
#     Remove all literal '/n' substrings and leading spaces from each line,
#     and remove any trailing empty (or whitespace-only) lines from the input string.
    
#     Args:
#         s (str): The input string.
    
#     Returns:
#         str: The cleaned string.
#     """
#     # Remove all literal occurrences of "/n"
#     s = s.replace("/n", "")
    
#     # Split the string into lines.
#     lines = s.splitlines()
    
#     # Remove trailing empty or whitespace-only lines.
#     while lines and not lines[-1].strip():
#         lines.pop()
    
#     # Remove leading spaces from each line.
#     lines = [line.lstrip() for line in lines]
    
#     # Reassemble the string with newline characters.
#     return "\n".join(lines)

In [12]:
# def clean_square_brackets(text):
#     """
#     Escape single square brackets in the text to prevent them from being treated as links in Markdown.

#     Args:
#         text (str): The input text containing square brackets.

#     Returns:
#         str: The text with escaped square brackets.
#     """
#     # Escape single square brackets
#     return re.sub(r'(?<!\\)\[([^\]]+)\]', r'\\[\1\\]', text)


In [ ]:
# def clean_sprite_tags_and_brackets(text):
#     """
#     Remove <sprite name=...> tags and escape square brackets in the text.

#     Args:
#         text (str): The input text.

#     Returns:
#         str: The cleaned text.
#     """
#     # Remove <sprite name=...> tags
#     text = re.sub(r'<sprite name=[^>]+>', '', text)
#     # Escape square brackets
#     return clean_square_brackets(text)

In [ ]:
# def clean_filename(filename: str) -> str:
#     """
#     Cleans a filename by:
#       - Removing disallowed Windows characters: < > : " / \\ | ? *
#       - Removing any spaces immediately preceding or following those characters.
#       - Replacing interior occurrences of a disallowed character (with its adjacent spaces) with an underscore.
#       - Removing disallowed characters at the beginning or end entirely.
    
#     Args:
#         filename (str): The original filename string.
    
#     Returns:
#         str: The cleaned filename.
#     """
#     # Define the set of disallowed characters.
#     invalid = r'[<>:"/\\|?*]'
    
#     # 1. Remove invalid characters (with any adjacent spaces) from the beginning.
#     filename = re.sub(r'^\s*(?:' + invalid + r'\s*)+', '', filename)
    
#     # 2. Remove invalid characters (with any adjacent spaces) from the end.
#     filename = re.sub(r'(?:\s*' + invalid + r')+\s*$', '', filename)
    
#     # 3. In the interior, replace any invalid character (with optional surrounding spaces)
#     #    with a single underscore.
#     filename = re.sub(r'\s*(' + invalid + r')\s*', '_', filename)
    
#     return remove_empty_lines(filename)

In [13]:
def check_first_row_and_column_duplicates(df, detailed=False):
    """
    Enhanced duplicate checker for first column values and all headers.
    
    Args:
        df (pd.DataFrame): Input DataFrame
        detailed (bool): If True, returns duplicate counts
        
    Returns:
        dict: {
            'column_duplicates': bool/Series,  # First column values
            'header_duplicates': bool/Series,  # All column headers
            'exact_header_duplicates': list   # List of duplicate header names
        }
    """
    results = {
        'column_duplicates': False,
        'header_duplicates': False,
        'exact_header_duplicates': []
    }
    
    # Check first column (values)
    first_col = df.iloc[:, 0]
    col_duplicates = first_col[first_col.duplicated(keep=False)]
    if detailed:
        results['column_duplicates'] = col_duplicates.value_counts().sort_values(ascending=False)
    else:
        results['column_duplicates'] = not col_duplicates.empty
    
    # Enhanced header check
    header_counts = pd.Series(df.columns).value_counts()
    dup_headers = header_counts[header_counts > 1]
    
    if not dup_headers.empty:
        results['header_duplicates'] = True
        results['exact_header_duplicates'] = dup_headers.index.tolist()
        if detailed:
            results['header_duplicates'] = dup_headers.sort_values(ascending=False)
    
    return results

In [14]:
def get_aliases(entry_data):
    """
    Extract and process aliases from specific columns in entry_data.

    Args:
        entry_data (dict): The dictionary containing row or sheet data to extract aliases.

    Returns:
        list: A list of processed aliases.
    """
    # Define the columns to check for aliases
    alias_columns = ['AKA', 'aliases', 'alias']

    # Check if any alias column exists in entry_data
    if not any(column in entry_data for column in alias_columns):
        return []  # Exit early if no alias columns are found

    aliases = set()

    # Extract and process aliases from the specified columns
    for column in alias_columns:
        if column in entry_data:
            # Split values by commas and strip whitespace
            raw_aliases = [alias.strip() for alias in str(entry_data[column]).split(',') if alias.strip()]
            for alias in raw_aliases:
                # Remove leading 'the' or 'a' (case-insensitive)
                alias = re.sub(r'^(the|a)\s+', '', alias, flags=re.IGNORECASE).strip()

                # Remove trailing 's or s
                if alias.endswith("'s"):
                    alias = alias[:-2]
                elif alias.endswith("s"):
                    alias = alias[:-1]

                aliases.add(alias)

    return list(aliases)

In [15]:
# def construct_game_content_dict(config_dict, master_url_dict):
#     """
#     Constructs the game_content_dict using multithreading for faster processing.

#     Args:
#         config_dict (dict): Configuration dictionary containing game information.
#         master_url_dict (dict): Dictionary containing URLs for each sheet.

#     Returns:
#         dict: The constructed game_content_dict.
#     """
#     game_content_dict = {}

#     def process_sheet(game, sheet, url):
#         """
#         Processes a single sheet and returns its content.

#         Args:
#             game (str): The game name.
#             sheet (str): The sheet name.
#             url (str): The URL of the sheet.

#         Returns:
#             tuple: (game, sheet, sheet_content_dict)
#         """
#         try:
#             entry_dict = initial_content_dict_from_url(url)
#             sheet_content_dict = {}

#             for entry, content in entry_dict.items():
#                 # Clean the entry name for use as a key
#                 clean_entry = clean_filename(entry)
#                 empty_lines_removed_entry = remove_empty_lines(entry)

#                 # Step 1: Clean the content values
#                 cleaned_content = {
#                     key: clean_sprite_tags_and_brackets(str(value)) if isinstance(value, str) else value
#                     for key, value in content.items()
#                 }

#                 # Step 2: Capture 'AKA', 'aliases', or 'alias' values
#                 aliases = []
#                 for key in ['AKA', 'aliases', 'alias', 'search keys']:
#                     if key in content:
#                         # Split values by commas and strip whitespace
#                         aliases += [val.strip() for val in str(content[key]).split(',') if val.strip()]
# 
#                 # Step 3: Add 'search_keys' key
#                 search_keys = [empty_lines_removed_entry] + aliases
#                 enhanced_keys = enhance_search_keys(search_keys)

#                 # Step 4: Populate sheet_content_dict with the cleaned content
#                 sheet_content_dict[empty_lines_removed_entry] = {
#                     'title': empty_lines_removed_entry.strip(),  # Strip title
#                     'link': f'{game}/{sheet}/{clean_entry}'.strip(),  # Strip link
#                     'content': cleaned_content,
#                     'search_keys': enhanced_keys,
#                     'references': [],
#                 }

#             return game, sheet, sheet_content_dict
#         except Exception as e:
#             print(f"Error processing sheet '{sheet}' for game '{game}': {e}")
#             return game, sheet, {}

#     # Use ThreadPoolExecutor to process sheets concurrently
#     with ThreadPoolExecutor() as executor:
#         futures = []
#         for game in config_dict['games']:
#             if game in master_url_dict:
#                 for sheet, url in master_url_dict[game].items():
#                     futures.append(executor.submit(process_sheet, game, sheet, url))

#         # Collect results as they complete
#         for future in as_completed(futures):
#             game, sheet, sheet_content_dict = future.result()
#             if game not in game_content_dict:
#                 game_content_dict[game] = {}
#             game_content_dict[game][sheet] = sheet_content_dict

#     return game_content_dict

In [16]:
# def construct_initial_meta_dict(master_url_dict):
#     """
#     Constructs the meta_lore_dict using multithreading for faster processing.

#     Args:
#         master_url_dict (dict): Dictionary containing URLs for each sheet in the 'Meta Lore' category.

#     Returns:
#         dict: The constructed meta_lore_dict.
#     """
#     meta_lore_dict = {}

#     def process_meta_sheet(sheet, url):
#         """
#         Processes a single meta sheet and returns its content.

#         Args:
#             sheet (str): The sheet name.
#             url (str): The URL of the sheet.

#         Returns:
#             tuple: (sheet, sheet_meta_dict)
#         """
#         try:
#             # Read CSV with no header
#             df = pd.read_csv(url, header=None)

#             # Check if DataFrame has at least two rows (header + at least one data row)
#             if len(df) > 1:
#                 first_row = df.iloc[1]  # Get the first row after the header

#                 # Extract aliases: non-empty values from cells starting with 'AKA' (case-insensitive)
#                 if str(first_row[0]).strip().upper() == 'AKA':
#                     aliases = [
#                         str(x).strip() for x in first_row[1:] if pd.notna(x) and str(x).strip()
#                     ]
#                     # Exclude the 'AKA' row from the entry list
#                     entry_list = df.iloc[2:, 0].tolist()  # Start from the second row after the header
#                 else:
#                     aliases = []
#                     entry_list = df.iloc[1:, 0].tolist()  # Include all rows after the header
#             else:
#                 aliases = []
#                 entry_list = []

#             # Construct the sheet meta dictionary
#             sheet_meta_dict = {
#                 'title': sheet,
#                 'search_keys': [sheet] + aliases,
#                 'link': f'Meta Lore/{sheet}',
#                 'references': [],
#             }

#             for entry in entry_list:
#                 sheet_meta_dict[entry] = {
#                     'title': entry,
#                     'link': f'Meta Lore/{sheet}/{entry}',
#                     'references': [],
#                 }

#             return sheet, sheet_meta_dict
#         except Exception as e:
#             print(f"Error processing meta sheet '{sheet}': {e}")
#             return sheet, {}

#     # Use ThreadPoolExecutor to process sheets concurrently
#     with ThreadPoolExecutor() as executor:
#         futures = []
#         for sheet, url in master_url_dict['Meta Lore'].items():
#             futures.append(executor.submit(process_meta_sheet, sheet, url))

#         # Collect results as they complete
#         for future in as_completed(futures):
#             sheet, sheet_meta_dict = future.result()
#             meta_lore_dict[sheet] = sheet_meta_dict

#     return meta_lore_dict

In [17]:
def search_keys_in_strings(search_keys, input_strings):
    """
    Checks if any of the search_keys exist as a string or substring in any of the input strings.

    Args:
        search_keys (list): List of search keys to check.
        input_strings (list): List of strings to search within.

    Returns:
        bool: True if any search key exists as a string or substring in any of the input strings, False otherwise.
    """
    for input_string in input_strings:
        for key in search_keys:
            if key in input_string:
                return True
    return False

In [18]:
def compare_and_update_references(dict_1, dict_2):
    """
    Compares search keys and content between two dictionaries and updates their references.

    Args:
        dict_1 (dict): The first dictionary with 'search_keys', 'content', and 'references'.
        dict_2 (dict): The second dictionary with 'search_keys', 'content', and 'references'.

    Returns:
        None: Updates the 'references' key in both dictionaries in place.
    """
    # Step 1: Extract search keys and convert to strings
    search_keys_1 = set(str(key).strip() for key in dict_1.get('search_keys', []))
    search_keys_2 = set(str(key).strip() for key in dict_2.get('search_keys', []))

    # Step 2: Check for overlap in search keys
    search_key_overlap = not search_keys_1.isdisjoint(search_keys_2)

    # Step 3: Extract strings to search in and handle lists or strings
    def extract_strings(content):
        if isinstance(content, list):
            return [str(value) for value in content]
        elif isinstance(content, str):
            return [content]
        return []

    strings_to_search_in_1 = extract_strings(dict_1.get('content', {}))
    strings_to_search_in_2 = extract_strings(dict_2.get('content', {}))

    # Step 4: Use search_keys_in_strings to compare content
    result_1 = search_keys_in_strings(search_keys_1, strings_to_search_in_2)
    result_2 = search_keys_in_strings(search_keys_2, strings_to_search_in_1)

    # Step 5: Combine conditions
    if search_key_overlap or result_1 or result_2:
        # Update references in dict_1
        references_1 = set(dict_1.get('references', []))
        references_1.add(dict_2.get('link', '').strip())
        dict_1['references'] = list(references_1)

        # Update references in dict_2
        references_2 = set(dict_2.get('references', []))
        references_2.add(dict_1.get('link', '').strip())
        dict_2['references'] = list(references_2)

In [19]:
def process_all_game_sheets(game_content_dict):
    """
    Cross-reference all entry_dicts across all games and sheets in game_content_dict.
    """
    all_entry_dicts = []
    for game, sheets in game_content_dict.items():
        for sheet, entries in sheets.items():
            for entry_key, entry in entries.items():
                # Ensure the entry itself is a dictionary
                if isinstance(dict_1.get('content', {}), dict) and isinstance(dict_2.get('content', {}), dict):
                    compare_and_update_references(dict_1, dict_2)
                # Add the entry to the list for comparison
                all_entry_dicts.append(entry)

    print(f"Total entries to compare: {len(all_entry_dicts)}")

    # Generate all pairs of entries for comparison
    entry_pairs = list(combinations(all_entry_dicts, 2))
    print(f"Total pairs to compare: {len(entry_pairs)}")

    for dict_1, dict_2 in entry_pairs:
        try:
            # Compare and update references only if 'content' is a dictionary
            compare_and_update_references(dict_1, dict_2)
        except Exception as e:
            print(f"Error comparing '{dict_1.get('link', 'Unknown')}' and '{dict_2.get('link', 'Unknown')}': {e}")

In [20]:
def dict_to_markdown(data, vault_path):
    """
    Convert nested dictionary structure to Markdown files.

    Args:
        data (dict): Nested dictionary in the format:
            {game1: {sheet1: {entry1: {'title': title, 'link': link, 
                    'content': {key1:value1, key2:value2,...}, 'search_keys': [...], 'references': [...]}}}, ...}
        vault_path (str): Root directory where files should be saved
    """
    for game, game_data in data.items():
        for sheet, sheet_data in game_data.items():
            for entry_key, entry_data in sheet_data.items():
                # Ensure the entry itself is a dictionary
                if not isinstance(entry_data, dict):
                    print(f"Warning: Entry data is not a dictionary in {game} -> {sheet} -> {entry_key}: {type(entry_data).__name__}")
                    continue

                # Process 'link' only if it's a string
                link = entry_data.get('link', '')
                if not isinstance(link, str):
                    print(f"Warning: 'link' is not a string in {game} -> {sheet} -> {entry_key}: {type(link).__name__}")
                    continue

                # Create the file path
                file_path = Path(vault_path) / f"{link}.md"
                file_path.parent.mkdir(parents=True, exist_ok=True)

                # Write the Markdown file
                with open(file_path, 'w', encoding='utf-8') as f:
                    # Process 'content' only if it's a dictionary
                    content = entry_data.get('content', {})
                    if isinstance(content, dict):
                        for key, value in content.items():
                            if isinstance(value, str) and '\n' in value:
                                value = value.replace('\n', '\n  ')
                            f.write(f"**{key}**: {value}\n\n")

                    # Process 'search_keys' only if it's a list or string
                    search_keys = entry_data.get('search_keys', [])
                    if isinstance(search_keys, (list, str)):
                        f.write(f"**search keys**: {', '.join(search_keys) if isinstance(search_keys, list) else search_keys}\n\n")

                    # Process 'references' only if it's a list
                    references = entry_data.get('references', [])
                    if isinstance(references, list):
                        f.write("\n## References\n")
                        for ref in references:
                            f.write(f"- [[{ref}]]\n")

In [21]:
# from concurrent.futures import ThreadPoolExecutor
# from threading import Lock

# def dict_to_markdown(data, vault_path, batch_size=100):
#     """
#     Convert nested dictionary structure to Markdown files using multithreading with batch processing.

#     Args:
#         data (dict): Nested dictionary in the format:
#             {game1: {sheet1: {entry1: {'title': title, 'link': link, 
#                     'content': {key1:value1, key2:value2,...}, 'search_keys': [...], 'references': [...]}}}, ...}
#         vault_path (str): Root directory where files should be saved.
#         batch_size (int): Number of entries to process in each batch.
#     """
#     dir_lock = Lock()  # Lock for synchronizing directory creation

#     def process_batch(batch, vault_path):
#         """
#         Process a batch of entries and write them to Markdown files.

#         Args:
#             batch (list): List of entry dictionaries.
#             vault_path (str): Root directory where files should be saved.
#         """
#         for entry_data in batch:
#             # Get entry details
#             title = entry_data.get('title', 'Untitled')
#             link = entry_data.get('link', '')
#             content = entry_data.get('content', {})
#             search_keys = entry_data.get('search_keys', [])
#             references = entry_data.get('references', [])

#             # Create full file path
#             file_path = Path(vault_path) / f"{link}.md"

#             # Ensure parent directories exist
#             parent_dir = file_path.parent
#             with dir_lock:  # Synchronize directory creation
#                 if not parent_dir.exists():
#                     parent_dir.mkdir(parents=True, exist_ok=True)

#             # Write markdown content
#             with open(file_path, 'w', encoding='utf-8') as f:
#                 # Write each content key-value pair
#                 for key, value in content.items():
#                     # Handle multiline values by indenting subsequent lines
#                     if isinstance(value, str) and '\n' in value:
#                         value = value.replace('\n', '\n  ')
#                     f.write(f"**{key}**: {value}\n\n")

#                 # Write search keys
#                 if search_keys:
#                     f.write(f"**search keys**: {', '.join(search_keys)}\n\n")

#                 # Write references at the end
#                 if references:
#                     f.write("\n## References\n")
#                     for ref in references:
#                         f.write(f"- [[{ref}]]\n")

#     # Flatten the nested dictionary into a list of entries
#     entries = [
#         entry_data
#         for game_data in data.values()
#         for sheet_data in game_data.values()
#         for entry_data in sheet_data.values()
#     ]

#     # Process entries in batches using ThreadPoolExecutor
#     with ThreadPoolExecutor() as executor:
#         futures = []
#         for i in range(0, len(entries), batch_size):
#             batch = entries[i:i + batch_size]
#             futures.append(executor.submit(process_batch, batch, vault_path))

#         # Wait for all threads to complete
#         for future in futures:
#             future.result()

In [24]:
# def main():
script_path = Path(get_script_directory())
master_path = script_path.parent
vault_path = master_path / 'Obsidian Vault'

delete_all_except_obsidian(vault_path)

config_path=script_path / 'config.json'
sheets_path=script_path / 'sheets.json'

# config_dict=load_json(config_path)
# sheets_dict=load_json(sheets_path)

# master_url_dict=construct_master_url_dict(sheets_dict)

# game_content_dict=construct_game_content_dict(config_dict, master_url_dict)
# meta_lore_dict=construct_initial_meta_dict(master_url_dict)

# dict_to_markdown(game_content_dict, vault_path)

In [25]:
config_dict=load_json(config_path)
sheets_dict=load_json(sheets_path)

In [26]:
master_url_dict=construct_master_url_dict(sheets_dict)

In [ ]:
# game_content_dict=construct_game_content_dict(config_dict, master_url_dict)

In [ ]:
# meta_lore_dict=construct_initial_meta_dict(master_url_dict)

In [ ]:
# game_content_dict['Cultist Simulator']

In [ ]:
# meta_lore_dict

In [ ]:
# master_dict=game_content_dict
# master_dict['Meta Lore']=meta_lore_dict

In [ ]:
# master_dict['Meta Lore']

In [ ]:
# process_all_game_sheets(master_dict)

In [ ]:
# dict_to_markdown(master_dict, vault_path)

In [ ]:
# game_content_dict["Meta Lore"]["Principles"]

In [ ]:
# search_keys = ["Forge of Days"]
# enhanced_keys = enhance_search_keys(search_keys)
# print("Enhanced Search Keys:", enhanced_keys)

In [ ]:
# def create_smaller_game_content_dict(original_dict, max_entries=1000):
#     """
#     Create a smaller version of the game_content_dict for testing.
    
#     Args:
#         original_dict (dict): The original game_content_dict.
#         max_entries (int): The maximum number of entries to include in the smaller dictionary.
    
#     Returns:
#         dict: A smaller version of the game_content_dict.
#     """
#     smaller_dict = {}
#     included_entries = 0

#     # Ensure the required entries are included
#     required_entries = [
#         ("Cultist Simulator", "Expeditions", "Cater & Hero Limited"),
#         ("Cultist Simulator", "Ingredients", "Vital Pigment")
#     ]

#     # Add required entries first
#     for game, sheet, entry in required_entries:
#         if game in original_dict and sheet in original_dict[game] and entry in original_dict[game][sheet]:
#             if game not in smaller_dict:
#                 smaller_dict[game] = {}
#             if sheet not in smaller_dict[game]:
#                 smaller_dict[game][sheet] = {}
#             smaller_dict[game][sheet][entry] = original_dict[game][sheet][entry]
#             included_entries += 1

#     # Iterate over the original dictionary to add more entries
#     for game, sheets in original_dict.items():
#         if game not in smaller_dict:
#             smaller_dict[game] = {}
#         for sheet, entries in sheets.items():
#             if sheet not in smaller_dict[game]:
#                 smaller_dict[game][sheet] = {}
#             for entry, data in entries.items():
#                 # Skip if we've already added the required entries or reached the max limit
#                 if included_entries >= max_entries:
#                     break
#                 if entry not in smaller_dict[game][sheet]:
#                     smaller_dict[game][sheet][entry] = data
#                     included_entries += 1
#             if included_entries >= max_entries:
#                 break
#         if included_entries >= max_entries:
#             break

#     return smaller_dict

In [ ]:
# # Example usage
# smaller_game_content_dict = create_smaller_game_content_dict(game_content_dict, max_entries=10000)
# print(json.dumps(smaller_game_content_dict, indent=4))

In [ ]:
# game_content_dict['Book of Hours']['Assistance']['Barber']

In [ ]:
# game_content_dict.keys()

In [ ]:
# dict_1 = game_content_dict['Cultist Simulator']['Expeditions']['Cater & Hero Limited']
# dict_2 = game_content_dict['Cultist Simulator']['Ingredients']['Vital Pigment']
# compare_and_update_references(dict_1, dict_2)

In [ ]:
# meta_lore_dict['Principles']

In [ ]:
os.path.dirname(script_path)

In [ ]:
# print(game_content_dict)

In [ ]:
# print(meta_lore_dict)

In [ ]:
# game_content_dict['Book of Hours']['Books'].keys()

In [ ]:
# import requests
# import pandas as pd

# def check_index_duplicates(df, detailed=False):
#     """
#     Check only for duplicates in the first column (indices)
    
#     Args:
#         df: pandas DataFrame
#         detailed: If True, returns value counts
        
#     Returns:
#         Series if detailed=True, else bool
#     """
#     first_col = df.iloc[:, 0]
#     duplicates = first_col[first_col.duplicated(keep=False)]
    
#     if detailed:
#         return duplicates.value_counts().sort_values(ascending=False)
#     return not duplicates.empty

# # Main processing loop
# for game in config_dict['games']:
#     for sheet, url in master_url_dict[game].items():
#         try:
#             # Load the DataFrame directly (no need for header check)
#             df = pd.read_csv(url)
            
#             # Check only for index duplicates
#             duplicates = check_index_duplicates(df, detailed=True)
            
#             # Print results if duplicates found
#             if not duplicates.empty:
#                 print(f'\n[INDICES] Duplicates found in {game} - {sheet}:')
#                 print(duplicates.to_string())
                
#                 # Optional: Show the headers for reference
#                 print("\nCurrent headers:", df.columns.tolist())
                
#         except Exception as e:
#             print(f"\nError processing {game} - {sheet}: {str(e)}")

In [ ]:
# df = pd.read_csv(master_url_dict['Book of Hours']['Crafting Recipe'])
# print("Actual headers:", df.columns.tolist())

In [ ]:
# game_content_dict
#     # print(game)
#     # print(master_url_dict[game])

In [ ]:
# boh_memory_url=master_url_dict['Book of Hours']['Memories']
# initial_content_dict_from_url(boh_memory_url)

In [ ]:
# # Create output directory relative to script location
# script_dir = Path(get_script_directory())
# output_dir = script_dir.parent / "Obsidian/markdown_files"
# output_dir.mkdir(exist_ok=True)

# def row_value_pairs(row):
#     """Convert a pandas row to Obsidian-friendly markdown format"""
#     content = []
    
#     # Get the first column's value (regardless of other columns)
#     first_col_name = row.index[0]  # Name of the first column
    
#     # Add other columns as key-value pairs (skip the first column)
#     # content.append(f"- **Type**: {type}")
#     for col, val in row.items():
#         if col == first_col_name:  # Skip the first column (already used as heading)
#             content.append(f"- **Type**: {col}")
#         if pd.notna(val):
#             clean_val = str(val).strip().replace('\r\n', '\n').replace('\n', '<br>')
#             content.append(f"- **{col}**: {clean_val}")
    
#     return "\n".join(content)

# def md_files_from_df(df):
#     # Write each row to a markdown file
#     for index, row in df.iterrows():
#         # Create safe filename (remove special chars)
#         safe_filename = row_first_value(row).replace(':', ' -') + '.md'
#         filepath = output_dir / safe_filename
#         # print(f'filepath: {filepath}')
        
#         try:
#             with open(filepath, 'w', encoding='utf-8') as f:
#                 f.write(row_value_pairs(row))
#             print(f"✓ Created: {filepath.relative_to(output_dir)}")
#         except Exception as e:
#             print(f"✗ Error writing {filepath.name}: {str(e)}")

In [ ]:
''' 
game_content_dict={'game':
                    {'sheet': 
                        {'entry': {
                            'title': 'Title',
                            'link': 'Link',
                            'content': {
                                'key1': 'value1',
                                'key2': 'value2'
                            },
                            'search_keys': ['key1', 'key2'],
                            'references': ['ref1', 'ref2']
                        }
                    }
}

game_content_dict['game']=
    {'sheet': {
        'entry': 
            {
            'title': 'Title',
            'link': 'Link',
            'content': {
                'key1': 'value1',
                'key2': 'value2'
            },
            'search_keys': ['key1', 'key2'],
            'references': ['ref1', 'ref2']
        }
    }
}

meta_lore_dict=
    {'sheet':
        {'entry': {
            'title': 'Title',
            'link': 'Link',
            'references': ['ref1', 'ref2']
        }
    }
}

complete_dict= {
    'game1': {
        'sheet1': {
            'title': '<sheet1>'
            ,'link': '<game1>/<sheet1>'
            ,'search_keys': ['key1', 'key2']
            ,'references': ['ref1', 'ref2']
            ,'content': {
                'entry1': {
                    'title': 'Title'
                    ,'link': 'Link'
                    ,'search_keys': ['key1', 'key2']
                    ,'references': ['ref1', 'ref2']
                    ,'content': {
                        'key1': 'value1'
                        ,'key2': 'value2'
                    }
                }
                ,'entry2': {
                    'title': 'Title'
                    ,'link': 'Link'
                    ,'search_keys': ['key1', 'key2']
                    ,'references': ['ref1', 'ref2']
                    ,'content': {
                        'key1': 'value1'
                        , 'key2': 'value2'
                    }
                }
            }
        }
        ,'sheet2': {
            'title': '<sheet2>'
            ,'link': '<game1>/<sheet2>'
            ,'search_keys': ['key1', 'key2']
            ,'references': ['ref1', 'ref2']
            ,'content': {
                'entry1': {
                    'title': 'Title'
                    ,'link': 'Link'
                    ,'search_keys': ['key1', 'key2']
                    ,'references': ['ref1', 'ref2']
                    ,'content': {
                        'key1': 'value1'
                        ,'key2': 'value2'
                    }
                }
                ,'entry2': {
                    'title': 'Title'
                    ,'link': 'Link'
                    ,'search_keys': ['key1', 'key2']
                    ,'references': ['ref1', 'ref2']
                    ,'content': {
                        'key1': 'value1'
                        , 'key2': 'value2'
                    }
                }
            }
        }
    }
    ,'game2': {
        'sheet1':
        . 
        . 
        .
    }
    , 'meta1': {
        'sheet1':
        . 
        . 
        .
    }
}
'''



In [ ]:
# meta_lore_dict

In [27]:
def clean_square_brackets(text):
    """
    Escape single square brackets in the text to prevent them from being treated as links in Markdown.

    Args:
        text (str): The input text containing square brackets.

    Returns:
        str: The text with escaped square brackets.
    """
    return re.sub(r'(?<!\\)\[([^\]]+)\]', r'\\[\1\\]', text)

In [ ]:
# text = "This is [an example]"
# print(clean_square_brackets(text))
# # Output: "This is \\[an example\\]"

In [28]:
def clean_sprite_tags(text):
    """
    Remove <sprite name=...> tags from the text and ensure no double spaces.

    Args:
        text (str): The input text.

    Returns:
        str: The cleaned text.
    """
    # Remove <sprite name=...> tags
    text = re.sub(r'<sprite name=[^>]+>', '', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [ ]:
# text = "This is a <sprite name=example> test"
# print(clean_sprite_tags(text))
# # Output: "This is a test"

In [29]:
def clean_sprite_tags_and_brackets(text):
    """
    Remove <sprite name=...> tags and escape square brackets in the text.

    Args:
        text (str): The input text.

    Returns:
        str: The cleaned text.
    """
    text = clean_sprite_tags(text)  # Remove sprite tags
    return clean_square_brackets(text)  # Escape square brackets

In [30]:
# text = "This is a <sprite name=example> test [example]"
# print(clean_sprite_tags_and_brackets(text))
# # Output: "This is a test \\[example\\]"

In [31]:
def remove_empty_lines(s):
    """
    Remove all literal '/n' substrings and leading spaces from each line,
    and remove any empty (or whitespace-only) lines from the input string.

    Args:
        s (str): The input string.

    Returns:
        str: The cleaned string.
    """
    s = s.replace("/n", "")  # Remove literal '/n'
    lines = s.splitlines()  # Split into lines
    # Remove lines that are empty or contain only whitespace
    lines = [line.lstrip() for line in lines if line.strip()]
    return "\n".join(lines)  # Reassemble the string

In [32]:
# text = "  Line 1\n\n  Line 2  \n\n"
# print(remove_empty_lines(text))
# # Output:
# # "Line 1
# # Line 2"

In [33]:
def clean_filename(filename):
    """
    Cleans a filename by removing disallowed Windows characters and replacing them with underscores.

    Args:
        filename (str): The original filename string.

    Returns:
        str: The cleaned filename.
    """
    invalid = r'[<>:"/\\|?*]'
    filename = re.sub(r'^\s*(?:' + invalid + r'\s*)+', '', filename)  # Remove invalid chars at the start
    filename = re.sub(r'(?:\s*' + invalid + r')+\s*$', '', filename)  # Remove invalid chars at the end
    filename = re.sub(r'\s*(' + invalid + r')\s*', '_', filename)  # Replace invalid chars with underscores
    return remove_empty_lines(filename)  # Remove empty lines

In [34]:
# filename = "  invalid:file/name?*  "
# print(clean_filename(filename))
# # Output: "invalid_file_name"

In [35]:
def enhance_search_keys(search_keys, config):
    """
    Enhance search keys by:
    - Removing leading 'the', 'a', or 'an' and adding individual words for keys separated by delimiters.
    - Adding variations of words ending with 's' by removing the 's'.
    - Excluding specific words and digits from the enhanced keys.

    Args:
        search_keys (list): List of original search keys.
        config (dict): Configuration dictionary containing excluded words, digits, and delimiters.

    Returns:
        list: Enhanced list of search keys.
    """
    excluded_words = set(config.get("excluded_words", []))
    excluded_digits = set(config.get("excluded_digits", []))
    delimiters = "|".join(map(re.escape, config.get("delimiters", ["_", ":", "-", " "])))

    enhanced_keys = set()

    for key in search_keys:
        # Remove leading 'the', 'a', or 'an' (case-insensitive) and the separator after it
        key = re.sub(r'^(the|a|an)[{}]+'.format(delimiters), '', key, flags=re.IGNORECASE).strip()

        # Add the cleaned key to the enhanced keys
        enhanced_keys.add(key)

        # Split the key by delimiters
        parts = re.split(delimiters, key)
        for part in parts:
            if part and part.lower() not in excluded_words and part not in excluded_digits:
                enhanced_keys.add(part)

                # If the part ends with 's, add the version without 's
                if part.endswith("'s"):
                    enhanced_keys.add(part[:-2])
                # If the part ends with 's' (without the apostrophe), add the version without 's'
                elif part.endswith("s"):
                    enhanced_keys.add(part[:-1])

    return list(enhanced_keys)

In [36]:
# # Load the config file
# config_path = "g:\\My Drive\\Drive\\Gaming_Music_Comics_software\\Weather Factory\\Code\\config.json"
# with open(config_path, "r") as file:
#     config = json.load(file)

# # Example search keys
# search_keys = ["The_Forge_of_Days", "Books", "123_Example"]

# # Enhance search keys using the config
# enhanced_keys = enhance_search_keys(search_keys, config)
# print("Enhanced Search Keys:", enhanced_keys)

In [37]:
def extract_sheet_content(url, sheet_name):
    """
    Reads a CSV file and constructs a content dictionary.
    Detects a row with the same name as the sheet title to extract aliases.

    Args:
        url (str): The URL or path to the CSV file.
        sheet_name (str): The name of the sheet.

    Returns:
        tuple: (content_dict, aliases)
    """
    df = pd.read_csv(url)
    print("Loaded DataFrame:")
    print(df)

    # Extract aliases and clean the DataFrame
    aliases = extract_aliases(df, sheet_name)

    # Construct the content dictionary
    content_dict = construct_content_dict(df)

    return content_dict, aliases


def extract_aliases(df, sheet_name):
    aliases = []
    print(f"Looking for aliases in sheet: {sheet_name}")
    print("First column values:", df.iloc[:, 0].values)
    
    if sheet_name in df.iloc[:, 0].values:
        aliases_row = df[df.iloc[:, 0] == sheet_name].iloc[0, 1:]
        aliases = [str(alias).strip() for alias in aliases_row if pd.notna(alias)]
        df.drop(df[df.iloc[:, 0] == sheet_name].index, inplace=True)
    
    print("Extracted aliases:", aliases)
    return aliases


def construct_content_dict(df):
    print("Constructing content dictionary...")
    print("DataFrame before processing:")
    print(df)
    
    content_dict = {}
    for _, row in df.iterrows():
        if pd.notna(row.iloc[0]):  # First column as key
            index_value = row.iloc[0]
            content_dict[index_value] = {}
            for i, val in enumerate(row.iloc[1:]):
                if pd.notna(val):
                    col_name = df.columns[i + 1]
                    content_dict[index_value][col_name] = val
    
    print("Constructed content dictionary:", content_dict)
    return content_dict

In [38]:
# import pandas as pd

# # Sample DataFrame
# data = {
#     "Column1": ["Sheet1", "Entry1", "Entry2"],
#     "Column2": ["Alias1", "Value1", "Value2"],
#     "Column3": ["Alias2", "Value3", "Value4"]
# }
# df = pd.DataFrame(data)

# # Test `extract_aliases`
# sheet_name = "Sheet1"
# aliases = extract_aliases(df, sheet_name)
# print("Aliases:", aliases)
# print("DataFrame after removing aliases row:")
# print(df)

In [39]:
def construct_unified_dict(config_dict, master_url_dict):
    """
    Constructs a unified dictionary for both games and meta entries.

    Args:
        config_dict (dict): Configuration dictionary containing game and meta information.
        master_url_dict (dict): Dictionary containing URLs for each sheet.

    Returns:
        dict: The constructed unified dictionary.
    """
    unified_dict = {}

    for category in config_dict.get('games', []) + config_dict.get('meta', []):
        if category in master_url_dict:
            category_dict = {}
            for sheet_name, url in master_url_dict[category].items():
                try:
                    # Use the existing helper function to process the sheet
                    content, aliases = extract_sheet_content(url, sheet_name)
                    sheet_dict = {
                        'title': sheet_name,
                        'link': f"{'Meta Lore' if category in config_dict.get('meta', []) else category}/{sheet_name}",
                        'search_keys': [sheet_name] + aliases,
                        'references': [],
                        'content': content
                    }
                    category_dict[sheet_name] = sheet_dict
                except Exception as e:
                    print(f"Error processing sheet '{sheet_name}' in category '{category}': {e}")
                    # Create an empty sheet dictionary if there's an error
                    category_dict[sheet_name] = {
                        'title': sheet_name,
                        'link': f"{'Meta Lore' if category in config_dict.get('meta', []) else category}/{sheet_name}",
                        'search_keys': [sheet_name],
                        'references': [],
                        'content': {}
                    }
            unified_dict[category] = category_dict

    return unified_dict

In [40]:
master_url_dict

{'Book of Hours': {'Memories': 'https://docs.google.com/spreadsheets/d/1p1aWr5N0GXdATgP9jM9pB_lL5HHsKEaQDU8zp0h7GMM/export?format=csv&gid=57430724',
  'Consider Masterlist': 'https://docs.google.com/spreadsheets/d/1p1aWr5N0GXdATgP9jM9pB_lL5HHsKEaQDU8zp0h7GMM/export?format=csv&gid=432406626',
  'Books': 'https://docs.google.com/spreadsheets/d/1p1aWr5N0GXdATgP9jM9pB_lL5HHsKEaQDU8zp0h7GMM/export?format=csv&gid=161140297',
  'Skills': 'https://docs.google.com/spreadsheets/d/1p1aWr5N0GXdATgP9jM9pB_lL5HHsKEaQDU8zp0h7GMM/export?format=csv&gid=887548006',
  'Wisdom Branch': 'https://docs.google.com/spreadsheets/d/1p1aWr5N0GXdATgP9jM9pB_lL5HHsKEaQDU8zp0h7GMM/export?format=csv&gid=1238682685',
  'Sustenance': 'https://docs.google.com/spreadsheets/d/1p1aWr5N0GXdATgP9jM9pB_lL5HHsKEaQDU8zp0h7GMM/export?format=csv&gid=652283921',
  'Inedible Cooking Ingredient': 'https://docs.google.com/spreadsheets/d/1p1aWr5N0GXdATgP9jM9pB_lL5HHsKEaQDU8zp0h7GMM/export?format=csv&gid=126757897',
  'Beverages': 'http

In [41]:
unified_dict = construct_unified_dict(config_dict, master_url_dict)

Loaded DataFrame:
                             Unnamed: 0  Secret Histories Level  \
0                  Cater & Hero Limited                     2.0   
1   Congregation of St Felix of Schuren                     2.0   
2                   Forgotten Mithraeum                     2.0   
3                                   NaN                     NaN   
4                         Crowkiss Hill                     4.0   
5                         Gladwwyn Lake                     4.0   
6                      Keglin's Scratch                     4.0   
7                          Lockwood Fen                     4.0   
8                      Chateau Raveline                     6.0   
9                           Orthos Wood                     6.0   
10          The Vanderschaaf Collection                     6.0   
11                  Key-Hunter's Garret                     6.0   
12                     Caves of Candles                     8.0   
13         Grunewald's Permanent Circus     

In [ ]:
# Create and save the sample DataFrame to a CSV file for testing
csv_path = "sample_sheet.csv"
data = {
    "Column1": ["Sheet1", "Entry1", "Entry2"],
    "Column2": ["Alias1", "Value1", "Value2"],
    "Column3": ["Alias2", "Value3", "Value4"]
}
pd.DataFrame(data).to_csv(csv_path, index=False)

In [ ]:
content_dict, aliases = extract_sheet_content(csv_path, "Sheet1")
print("Content Dictionary:", content_dict)
print("Aliases:", aliases)

In [ ]:
# Load configuration and sheets data
config_dict = load_json('config.json')
sheets_dict = load_json('sheets.json')

# Print to verify
print("Config Dictionary:", config_dict)
print("Sheets Dictionary:", sheets_dict)

In [ ]:
# Construct master URL dictionary
master_url_dict = construct_master_url_dict(sheets_dict)

# Print to verify
print("Master URL Dictionary:", json.dumps(master_url_dict, indent=4))

In [ ]:
# Select a specific sheet to test
category = list(master_url_dict.keys())[0]  # First category (e.g., "Cultist Simulator")
sheet_name = list(master_url_dict[category].keys())[0]  # First sheet in the category
url = master_url_dict[category][sheet_name]

In [ ]:
# Debug: Load and print the DataFrame
df = pd.read_csv(url)
print("DataFrame Loaded:")
print(df)

In [ ]:
# Select a specific sheet to test
category = list(master_url_dict.keys())[0]  # First category (e.g., "Cultist Simulator")
sheet_name = list(master_url_dict[category].keys())[0]  # First sheet in the category
url = master_url_dict['Book of Hours']['Memories']

In [ ]:
master_url_dict["Cultist Simulator"]["Hours"]

In [ ]:
url = master_url_dict["Cultist Simulator"]["Hours"]
hours = initial_content_dict_from_url(url)
print(hours["Mother of Ants"])

In [ ]:
get_aliases(hours)

In [ ]:
for category in config_dict.get('games', []) + config_dict.get('meta', []):
    if category in master_url_dict:
        category_dict = {
            'title': category,
            'type': 'game' if category in config_dict.get('games', []) else 'meta',
            'content': {}
        }
        for sheet_name, url in master_url_dict[category].items():
            try:
                # Use initial_content_dict_from_url to process the sheet
                raw_content = initial_content_dict_from_url(url)
            except Exception as e:
                print(f"Error processing sheet '{sheet_name}' in category '{category}': {e}")
                # Create an empty sheet dictionary if there's an error


In [42]:
def process_sheet(category, sheet_name, url):
    """
    Processes a single sheet and returns its content.

    Args:
        category (str): The category (game or meta).
        sheet_name (str): The name of the sheet.
        url (str): The URL of the sheet.

    Returns:
        tuple: (category, sheet_name, sheet_dict)
    """
    try:
        # Use the existing helper function to process the sheet
        raw_content = initial_content_dict_from_url(url)
        sheet_content = {}

        # Check if there's a row matching the sheet_name
        sheet_level_aliases = []
        if sheet_name in raw_content:
            # Extract aliases for the sheet-level search_keys
            sheet_level_aliases = get_aliases(raw_content[sheet_name])

        # Enrich each entry in the content
        for entry_name, entry_data in raw_content.items():
            # Clean the entry name for use as a key
            clean_entry_name = clean_filename(entry_name)

            # Extract aliases using get_aliases
            aliases = get_aliases(entry_data)

            # Construct the enriched entry structure
            enriched_entry = {
                'title': entry_name.strip(),
                'type': 'row',
                'link': f"{category}/{sheet_name}/{clean_entry_name}".strip(),
                'search_keys': [entry_name.strip()] + aliases,  # Add aliases to search_keys
                'references': [],
                'content': entry_data  # Keep the original key-value pairs
            }

            # Add the enriched entry to the sheet content
            sheet_content[entry_name] = enriched_entry

        # Construct the sheet dictionary
        sheet_dict = {
            'title': sheet_name,
            'type': 'sheet',
            'link': f"{category}/{sheet_name}",
            'search_keys': [sheet_name] + sheet_level_aliases,  # Add sheet-level aliases
            'references': [],
            'content': sheet_content,
        }

        return category, sheet_name, sheet_dict

    except Exception as e:
        # Log the error with traceback
        print(f"Error processing sheet '{sheet_name}' in category '{category}': {type(e).__name__}: {e}")
        print("Traceback:")
        traceback.print_exc()
        # Return an empty sheet dictionary in case of an error
        return category, sheet_name, {
            'title': sheet_name,
            'type': 'sheet',
            'link': f"{category}/{sheet_name}",
            'search_keys': [sheet_name],
            'references': [],
            'content': {}
        }

In [ ]:
def construct_unified_dict(config_dict, master_url_dict):
    """
    Constructs a unified dictionary for both games and meta entries using multithreading.

    Args:
        config_dict (dict): Configuration dictionary containing game and meta information.
        master_url_dict (dict): Dictionary containing URLs for each sheet.

    Returns:
        dict: The constructed unified dictionary.
    """
    from concurrent.futures import ThreadPoolExecutor, as_completed

    unified_dict = {}

    # Use ThreadPoolExecutor to process sheets concurrently
    with ThreadPoolExecutor() as executor:
        futures = []

        # Submit tasks for each sheet in each category
        for category in config_dict.get('games', []) + config_dict.get('meta', []):
            if category in master_url_dict:
                for sheet_name, url in master_url_dict[category].items():
                    futures.append(executor.submit(process_sheet, category, sheet_name, url))

        # Collect results as they complete
        for future in as_completed(futures):
            category, sheet_name, sheet_dict = future.result()
            if category not in unified_dict:
                unified_dict[category] = {}
            unified_dict[category][sheet_name] = sheet_dict

    return unified_dict

In [ ]:
# # Example usage for a row
# entry_data = {
#     # "AKA": "Alias1, Alias2",
#     # "aliases": "Alternate1, Alternate2",
#     "other_column": "Some value"
# }
# search_keys = ["OriginalKey"]
# entry_name = "Cater & Hero Limited"

# enhanced_keys = get_aliases(entry_data)
# print("Enhanced Search Keys:", enhanced_keys)


In [43]:
unified_dict = construct_unified_dict(config_dict, master_url_dict)

# Print a sample to verify
print(json.dumps(unified_dict, indent=4))

Loaded DataFrame:
                             Unnamed: 0  Secret Histories Level  \
0                  Cater & Hero Limited                     2.0   
1   Congregation of St Felix of Schuren                     2.0   
2                   Forgotten Mithraeum                     2.0   
3                                   NaN                     NaN   
4                         Crowkiss Hill                     4.0   
5                         Gladwwyn Lake                     4.0   
6                      Keglin's Scratch                     4.0   
7                          Lockwood Fen                     4.0   
8                      Chateau Raveline                     6.0   
9                           Orthos Wood                     6.0   
10          The Vanderschaaf Collection                     6.0   
11                  Key-Hunter's Garret                     6.0   
12                     Caves of Candles                     8.0   
13         Grunewald's Permanent Circus     

In [44]:
process_all_game_sheets(unified_dict)

UnboundLocalError: cannot access local variable 'dict_1' where it is not associated with a value

In [ ]:
script_path = Path(__file__).parent
config_path = script_path / 'config.json'
sheets_path = script_path / 'sheets.json'
vault_path = script_path.parent / 'Obsidian Vault'

# Load configuration and sheets data
config_dict = load_json(config_path)
sheets_dict = load_json(sheets_path)

# Construct master URL dictionary
master_url_dict = construct_master_url_dict(sheets_dict)

# Construct game content dictionary
# game_content_dict = construct_game_content_dict(config_dict, master_url_dict)

# Construct meta lore dictionary
# meta_lore_dict = construct_initial_meta_dict(master_url_dict)

# Combine into master dictionary
# master_dict = game_content_dict
# master_dict['Meta Lore'] = meta_lore_dict

# Process all game sheets for cross-references
# process_all_game_sheets(master_dict)

# # Export to Markdown files
# dict_to_markdown(master_dict, vault_path)

print("Execution completed. Markdown files saved to:", vault_path)